# Pre-processing Idaho Allocation data for WaDEQA upload.
Date Updated: 10/22/2020
Purpose:  To pre-process the Idaho data into one master file for simple DataFrame creation and extraction.  Working Idaho data for WaDEQA 2.0 is mostly composed of point of diversion data.

Useful Links to Data:
Point of Diversion (POD): (download spreadsheet)
https://data-idwr.opendata.arcgis.com/datasets/water-right-pods

Place of Use (PoU): Water Right (download the Zipped Files). See metadata in the View. Open WaterRightPOUs.dbf into Excel/load it into pandas Python 
https://data-idwr.opendata.arcgis.com/pages/gis-data#WaterRights

In [1]:
# Needed Libararies
import os
import numpy as np
import pandas as pd
import datetime
import geopandas as gpd # the library that lets us read in shapefiles
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [2]:
# Working Directory
workingDir = "G:/Shared drives/WaDE Data/Idaho/WaterAllocation/RawInputData"
os.chdir(workingDir)

In [3]:
#Input Files
FI_POD = "ID_Water_Right_PODs_input.xlsx"
FI_POU = "ID_Water_Right_PoUs_input.xlsx"
FI_ExtendedContactNames = "IdwrExtendedContactNamesWithFieldNames_input.xlsx"

## POD Sites Data

In [4]:
# Read in inputs, create dataframes.

df_ECN = pd.read_excel(FI_ExtendedContactNames)

dfinPOD = pd.read_excel(FI_POD)
dfinPOU = pd.read_excel(FI_POU)

dfinPOD = pd.merge(dfinPOD, dfinPOU, left_on='RightID_POD', right_on='RightID', how='left')
print(len(dfinPOD))
dfinPOD.head(3)

293138


,PointOfDiversionID_POD,BasinNumber_POD,Basis_POD,DataSource_POD,DiversionName_POD,DiversionType_POD,MetalTagNumber_POD,OBJECTID_POD,OverallMaxDiversionRate_POD,Owner_POD,PriorityDate_POD,RightID_POD,SequenceNumber_POD,Source_POD,SourceQualifier_POD,SpatialDataID_POD,SplitSuffix_POD,Status_POD,TributaryOf_POD,TributaryOfQualifier_POD,VersionNumber_POD,WaterDistrictNumber_POD,WaterRightNumber_POD,WRDocs_POD,WRMap_POD,WRReport_POD,X_POD,Y_POD,OID_,WaterRight,BasinNumbe,SequenceNu,SplitSuffi,Status,PriorityDa,DecreedDat,Owner,WaterUse,WaterUseCo,TotalAcres,AcreLimit,Source,SourceQual,TributaryO,Tributar_1,WaterDistr,LargePOU,RightID,PlaceOfUse,WRReport,WRDocs,WRMap,SHAPE_STAr,SHAPE_STLe,Longitude,Latitude
0,16,63,License,QQ,NaN,NaN,,234669,0.19,MAURICE MC CLUE,1989-06-21T00:00:00.000Z,22,10824,GROUND WATER,NaN,482579.0,,Active,NaN,NaN,0,TBD,63-10824,https://www.idwr.idaho.gov/apps/ExtSearch/Rela...,https://maps.idwr.idaho.gov/Map/IDWRlayout?Bas...,https://www.idwr.idaho.gov/apps/ExtSearch/Righ...,2.305278e+06,1.390599e+06,12304.0,63-10824,63.0,10824.0,,Active,1989-06-21 00:00:00,NaN,MAURICE MC CLUE,IRRIGATION,1.0,15.0,0.0,GROUND WATER,,,,TBD,0.0,22.0,42287.0,https://www.idwr.idaho.gov/apps/ExtSearch/Righ...,https://www.idwr.idaho.gov/apps/ExtSearch/Rela...,https://maps.idwr.idaho.gov/Map/IDWRlayout?Bas...,245680.919922,2028.984162,-116.416171,43.692670
1,122,67,License,QQ,NaN,NaN,NaN,28412,0.05,UNITED STATES OF AMERICA ACTING THROUGH,1992-05-27T00:00:00.000Z,130,7744,SPRING,NaN,50203.0,,Active,SINKS,NaN,0,TBD,67-7744,https://www.idwr.idaho.gov/apps/ExtSearch/Rela...,https://maps.idwr.idaho.gov/Map/IDWRlayout?Bas...,https://www.idwr.idaho.gov/apps/ExtSearch/Righ...,2.301183e+06,1.453358e+06,72324.0,67-7744,67.0,7744.0,,Active,1992-05-27 00:00:00,NaN,UNITED STATES OF AMERICA ACTING THROUGH,STOCKWATER,4.0,0.0,0.0,SPRING,,SINKS,,TBD,0.0,130.0,249948.0,https://www.idwr.idaho.gov/apps/ExtSearch/Righ...,https://www.idwr.idaho.gov/apps/ExtSearch/Rela...,https://maps.idwr.idaho.gov/Map/IDWRlayout?Bas...,149233.338867,1546.339554,-116.490469,44.254309
2,160,63,License,QQ,NaN,NaN,NaN,7508,0.13,JAMES P MERTZ,1990-02-28T00:00:00.000Z,175,11206,GROUND WATER,NaN,8421.0,,Active,NaN,NaN,0,NWD,63-11206,https://www.idwr.idaho.gov/apps/ExtSearch/Rela...,https://maps.idwr.idaho.gov/Map/IDWRlayout?Bas...,https://www.idwr.idaho.gov/apps/ExtSearch/Righ...,2.268087e+06,1.383476e+06,68330.0,63-11206,63.0,11206.0,,Active,1990-02-28 00:00:00,NaN,RICHARD A SYMMS,COMMERCIAL,10.0,0.0,0.0,GROUND WATER,,,,NWD,0.0,175.0,236113.0,https://www.idwr.idaho.gov/apps/ExtSearch/Righ...,https://www.idwr.idaho.gov/apps/ExtSearch/Rela...,https://maps.idwr.idaho.gov/Map/IDWRlayout?Bas...,163543.566406,1617.673427,-116.874100,43.616130


In [ ]:
# Update datatype of Priority Date to fit WaDE 2.0 structure

dfinPOD['PriorityDate_POD'] = pd.to_datetime(dfinPOD['PriorityDate_POD'])
dfinPOD['PriorityDate_POD'] = pd.to_datetime(dfinPOD["PriorityDate_POD"].dt.strftime('%m/%d/%Y'))

In [ ]:
# WaterSourceType

WaterSourceTypeDict = {
"River" : "Surface Water",
"Ground Water" : "Groundwater",
"Spring" :  "Groundwater",
"Lake" :  "Surface Water",
"Pond" :  "Surface Water", 
"Canal" :  "Surface Water",
"Creek" : "Surface Water",
"Fork" : "Surface Water",
"Waste Water" : "Reuse",
"Drain" : "Drain",
"Gluch" : "Surface Water",
"Reservoir" : "reservoir",
"Slough" : "Surface Water",
"Ditch" : "Surface Water",
"Channel" : "Surface Water",
"Dry" : "Surface Water"}

def assignWaterSourceType(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = 'Unspecified'
    elif colrowValue == "GROUND WATER":
        outList = "Groundwater"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        String1 = String1.title()  # change to title format
        list_of_words = String1.split()
        for i in list_of_words:
            if i in WaterSourceTypeDict.keys():
                outList = WaterSourceTypeDict[i]
            else:
                outList = 'Unspecified'
    return outList

dfinPOD['in_WaterSourceTypeCV'] = dfinPOD.apply(lambda row: assignWaterSourceType(row['Source_POD']), axis=1)
dfinPOD.head(3)

In [ ]:
# Sitetype

SiteTypeDict = {
"River" : "River",
"Spring" : "Spring",
"Lake" : "Lake",
"Pond" : "Pond", 
"Canal" : "Canal",
"Creek" : "Creek",
"Fork" : "Fork",
"Waste Water" : "Waste Water",
"Drain" : "Drain",
"Gluch" : "Gluch",
"Reservoir" : "Reservoir",
"Slough" : "Slough",
"Ditch" : "Ditch"
}

def assignSiteType(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = "Unspecified"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        String1 = String1.title()  # change to title format
        list_of_words = String1.split()
        for i in list_of_words:
            if i in SiteTypeDict.keys():
                outList = SiteTypeDict[i]
            else:
                outList = "Unspecified"
    return outList

dfinPOD['in_SiteTypeCV'] = dfinPOD.apply(lambda row: assignSiteType(row['Source_POD']), axis=1)
dfinPOD

In [ ]:
# Updating Owner names whose text was cut. Use extened contact name input.

ExtOwnerNameDict = pd.Series(df_ECN.StreetAddress.values, index = df_ECN.RightID).to_dict()

def updateExtndedOwner(valRightID, valOwner):
    if valOwner == '' or pd.isnull(valOwner):
        outList = ''
    else:
        try:
            outList = ExtOwnerNameDict[valRightID]
        except:
            outList = valOwner
    return outList

dfinPOD['in_AllocationOwner'] = dfinPOD.apply(lambda row: updateExtndedOwner(row['RightID_POD'], row['Owner_POD']), axis=1)
dfinPOD.head(3)

In [ ]:
dfPOD = pd.DataFrame()

# Water Source
dfPOD['in_Geometry'] = ""
dfPOD['in_GNISFeatureNameCV'] = ""
dfPOD['in_WaterQualityIndicatorCV'] = ""
dfPOD['in_WaterSourceName'] = dfinPOD['Source_POD']
dfPOD['in_WaterSourceNativeID'] = ""
dfPOD['in_WaterSourceTypeCV'] = dfinPOD['in_WaterSourceTypeCV']

# Site
dfPOD['in_CoordinateAccuracy'] = "Unspecified"
dfPOD['in_CoordinateMethodCV'] = dfinPOD['DataSource_POD']
dfPOD['in_County'] = "Unspecified"
dfPOD['in_EPSGCodeCV'] = ""
dfPOD['in_Geometry'] = ""
dfPOD['in_GNISCodeCV'] = ""
dfPOD['in_HUC12'] = ""
dfPOD['in_HUC8'] = ""
dfPOD['in_Latitude'] = dfinPOD['Y_POD']
dfPOD['in_Longitude'] = dfinPOD['X_POD']
dfPOD['in_NHDNetworkStatusCV'] = ""
dfPOD['in_NHDProductCV'] = ""
dfPOD['in_PODorPOUSite'] = "POD"
dfPOD['in_SiteName'] = dfinPOD['DiversionName_POD']
dfPOD['in_SiteNativeID'] = "POD" + dfinPOD['PointOfDiversionID_POD'].astype(str)
dfPOD['in_SitePoint'] = ""
dfPOD['in_SiteTypeCV'] = dfinPOD['in_SiteTypeCV']
dfPOD['in_StateCV'] = ""
dfPOD['in_USGSSiteID'] = ""

# Allocation Fact
dfPOD['in_AllocationApplicationDate'] = ""
dfPOD['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
dfPOD['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
dfPOD['in_AllocationBasisCV'] = dfinPOD['Basis_POD']
dfPOD['in_AllocationChangeApplicationIndicator'] = ""
dfPOD['in_AllocationCommunityWaterSupplySystem'] = ""
dfPOD['in_AllocationCropDutyAmount'] = ""
dfPOD['in_AllocationExpirationDate'] = ""
dfPOD['in_AllocationFlow_CFS'] = dfinPOD['OverallMaxDiversionRate_POD']
dfPOD['in_AllocationLegalStatusCV'] = ""
dfPOD['in_AllocationNativeID'] = dfinPOD['RightID_POD']
dfPOD['in_AllocationOwner'] = dfinPOD['in_AllocationOwner']
dfPOD['in_AllocationPriorityDate'] = dfinPOD['PriorityDate_POD']
dfPOD['in_AllocationSDWISIdentifierCV'] = ""
dfPOD['in_AllocationTimeframeEnd'] = ""
dfPOD['in_AllocationTimeframeStart'] = ""
dfPOD['in_AllocationTypeCV'] = dfinPOD['Status_POD']
dfPOD['in_AllocationVolume_AF'] = ""
dfPOD['in_BeneficialUseCategory'] = dfinPOD['WaterUse']
dfPOD['in_CommunityWaterSupplySystem'] = ""
dfPOD['in_CropTypeCV'] = ""
dfPOD['in_CustomerTypeCV'] = ""
dfPOD['in_DataPublicationDate'] = ""
dfPOD['in_DataPublicationDOI'] = ""
dfPOD['in_ExemptOfVolumeFlowPriority'] = ""
dfPOD['in_GeneratedPowerCapacityMW'] = ""
dfPOD['in_IrrigatedAcreage'] = ""
dfPOD['in_IrrigationMethodCV'] = ""
dfPOD['in_LegacyAllocationIDs'] = ""
dfPOD['in_PopulationServed'] = ""
dfPOD['in_PowerType'] = ""
dfPOD['in_PrimaryUseCategory'] = ""
dfPOD['in_VariableSpecificUUID'] = ""
dfPOD['in_WaterAllocationNativeURL'] = dfinPOD['WRDocs_POD']

print(len(dfPOD))
dfPOD.head(3)

## POU Site Data

In [ ]:
# Read in input, create dataframe.

df_ECN = pd.read_excel(FI_ExtendedContactNames)

dfinPOU = pd.read_excel(FI_POU)
print(len(dfinPOU))
dfinPOU.head(3)

In [ ]:
# Update datatype of Priority Date to fit WaDE 2.0 structure.

dfinPOU['PriorityDa'] = pd.to_datetime(dfinPOU['PriorityDa'])
dfinPOU['PriorityDa'] = pd.to_datetime(dfinPOU["PriorityDa"].dt.strftime('%m/%d/%Y'))
dfinPOU.head(3)

In [ ]:
# AllocationVolume_AF

def assignAllocationVolume_AF(A, B):
    A = float(A)
    B = float(B)
    if A > 0:
        outString = A
    else:
        outString = B
    return outString

dfinPOU['in_AllocationVolume_AF'] = dfinPOU.apply(lambda row: assignAllocationVolume_AF(row['AcreLimit'], row['TotalAcres']), axis=1)
dfinPOU.head(3)

In [ ]:
# WaterSourceType

WaterSourceTypeDict = {
"River" : "Surface Water",
"Ground Water" : "Groundwater",
"Spring" :  "Groundwater",
"Lake" :  "Surface Water",
"Pond" :  "Surface Water", 
"Canal" :  "Surface Water",
"Creek" : "Surface Water",
"Fork" : "Surface Water",
"Waste Water" : "Reuse",
"Drain" : "Drain",
"Gluch" : "Surface Water",
"Reservoir" : "Reservoir",
"Slough" : "Surface Water",
"Ditch" : "Surface Water",
"Channel" : "Surface Water",
"Dry" : "Surface Water"}

def assignWaterSourceType(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = 'Unspecified'
    elif colrowValue == "GROUND WATER":
        outList = "Groundwater"
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        String1 = String1.title()  # change to title format
        list_of_words = String1.split()
        for i in list_of_words:
            if i in WaterSourceTypeDict.keys():
                outList = WaterSourceTypeDict[i]
            else:
                outList = 'Unspecified'
    return outList

dfinPOU['in_WaterSourceTypeCV'] = dfinPOU.apply(lambda row: assignWaterSourceType(row['Source']), axis=1)
dfinPOU.head(3)

In [ ]:
# Updating Owner names whose text was cut. Use extened contact name input.

ExtOwnerNameDict = pd.Series(df_ECN.StreetAddress.values, index = df_ECN.RightID).to_dict()

def updateExtndedOwner(valRightID, valOwner):
    if valOwner == '' or pd.isnull(valOwner):
        outList = ''
    else:
#         valRightID = valRightID.strip()  # remove whitespace chars
        try:
            outList = ExtOwnerNameDict[valRightID]
        except:
            outList = valOwner
    return outList

dfinPOU['in_AllocationOwner'] = dfinPOU.apply(lambda row: updateExtndedOwner(row['RightID'], row['Owner']), axis=1)
dfinPOU.head(3)

In [ ]:
dfPOU = pd.DataFrame()

# Water Source
dfPOU['in_WaterQualityIndicatorCV'] = ""
dfPOU['in_WaterSourceName'] = dfinPOU['Source']
dfPOU['in_WaterSourceTypeCV'] = dfinPOU['in_WaterSourceTypeCV']

# Site
dfPOU['in_CoordinateAccuracy'] = "Unspecified"
dfPOU['in_CoordinateMethodCV'] = "Centroid"
dfPOU['in_County'] = "Unspecified"
dfPOU['in_Latitude'] = dfinPOU['Latitude']
dfPOU['in_Longitude'] = dfinPOU['Longitude']
dfPOU['in_PODorPOUSite'] = "POU"
dfPOU['in_SiteName'] = "Unspecified"
dfPOU['in_SiteTypeCV'] = "Unspecified"
dfPOU['in_SiteNativeID'] = "POU" + dfinPOU['PlaceOfUse'].astype(str)

# Allocation Fact
dfPOU['in_AllocationNativeID'] = dfinPOU['RightID']
dfPOU['in_AllocationOwner'] = dfinPOU['in_AllocationOwner']
dfPOU['in_AllocationPriorityDate'] = dfinPOU['PriorityDa']
dfPOU['in_AllocationVolume_AF'] = dfinPOU['in_AllocationVolume_AF']
dfPOU['in_BeneficialUseCategory'] = dfinPOU['WaterUse']
dfPOU['in_WaterAllocationNativeURL'] = dfinPOU['WRDocs']

print(len(dfPOU))
dfPOU.head(3)

## Concatenate and Export

In [ ]:
# Concatenate dataframes
frames = [dfPOD, dfPOU]
outdf = pd.concat(frames)
outdf = outdf.drop_duplicates()
print(len(outdf))

In [ ]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDEID_WS" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceName'] = outdf['in_WaterSourceName']
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = outdf['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A, B):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceName'] == A) & 
                                       (dfWaterSourceNativeID['in_WaterSourceTypeCV'] == B), 'in_WaterSourceNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

outdf['in_WaterSourceNativeID'] = outdf.apply(lambda row: retrieveWaterSourceNativeID( row['in_WaterSourceName'], row['in_WaterSourceTypeCV']), axis=1)
outdf

## Shapefile Data
- For attaching gemetry to csv inputs.

In [ ]:
# PoU Shapefile Data
# Shapefile input
ShapeFileInput = gpd.read_file('shapefile/ID_PoU.shp')
dfPoUshapetemp = pd.DataFrame(ShapeFileInput)
dfPoUshapetemp.head(1)

In [ ]:
columnsList = ['in_SiteNativeID', 'geometry']
dfPoUshape = pd.DataFrame(columns=columnsList)
dfPoUshape['in_SiteNativeID'] = "POU" + dfPoUshapetemp['PlaceOfUse'].astype(str)
dfPoUshape['geometry'] = dfPoUshapetemp['geometry']
dfPoUshape = dfPoUshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
dfPoUshape.head(3)

## Review and Export

In [ ]:
outdf.dtypes

In [ ]:
#Exporting to Finished File
outdf.to_csv('P_IdahoMaster.csv', index=False)  # The output
dfPoUshape.to_csv('P_idGeometry.csv', index=False) # The output geometry.